# User Variables
Please input the required variables below

In [1]:
%pip install suite2p

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import s2p_vis_func
import s2p_con_func
import s2p_plot_rois_funcs

import suite2p

No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally


In [3]:
ops = suite2p.default_ops()
ops['fs'] = 13 # sampling rate of recording, determines binning for cell detection
ops['tau'] = 1.25 # timescale of gcamp to use for deconvolution
ops['threshold_scaling'] =  [0.6, 0.8] # set however many desired threshold_scaling values you wish to test

#The user must now choose which threshold scaling
#value they prefer
desired_threshold_scaling = 0.8
fname = '59879-0 666um 200pwr 10xobj 4frm avg suc70 quin30-000'
fdir =  r'C:\Users\Alex_dl\Documents\GitHub\NAPE_imaging_postprocess\napeca_post\sample_data'
tseries_start_end = [0, 100]
rois_to_plot = 50

In [4]:
import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (18, 13),
    'ytick.major.left': True,
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

C:\Users\Alex_dl\anaconda3\envs\napeca_post\lib\site-packages\ipykernel_launcher.py:14: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("jet").copy()
  


In [5]:
db = {
    'data_path': [fdir],
}
print(db)

{'data_path': ['C:\\Users\\Alex_dl\\Documents\\GitHub\\NAPE_imaging_postprocess\\napeca_post\\sample_data']}


In [6]:
output_ops={}
output_ops['save_path'] = db['data_path'][0]
base_dir = output_ops['save_path']
for threshold_scaling_values in ops['threshold_scaling']:
    ops['threshold_scaling'] = threshold_scaling_values
    
    #deals with creating and saving outputs into correct folders
    output_ops['save_path'] = os.path.join(base_dir, f'threshold_scaling_{threshold_scaling_values}')
    ops['save_folder'] = f'threshold_scaling_{threshold_scaling_values}'
    output_ops = suite2p.run_s2p(ops=ops, db=db)
    
    output_fig_dir = os.path.join(base_dir, 's2p_analysis')

    #functions that output the Registration visualizations(only outputs once)
    if not os.path.exists(output_fig_dir):
        s2p_vis_func.check_exist_dir(output_fig_dir)
        s2p_vis_func.reg_ref_imgs(output_ops, output_fig_dir)
        s2p_vis_func.reg_displacement_vis(output_ops, output_fig_dir)
    
    #functions that output the ROI detections visualizations
    s2p_vis_func.roi_mask_vis(output_ops, output_fig_dir, threshold_scaling_values)
    s2p_vis_func.roi_trace_vis(output_ops, output_fig_dir, threshold_scaling_values)

    #functions that output the post-processed roi_plots based on the desired number of ROI's
    path_dict = s2p_plot_rois_funcs.define_paths_roi_plots(fdir, threshold_scaling_values, tseries_start_end, rois_to_plot, output_fig_dir)
    s2p_data_dict = s2p_plot_rois_funcs.load_s2p_data_roi_plots(path_dict)
    plot_vars = s2p_plot_rois_funcs.plotting_rois(s2p_data_dict, path_dict)
    s2p_plot_rois_funcs.template_init(plot_vars, s2p_data_dict)
    s2p_plot_rois_funcs.contour_plot(s2p_data_dict, path_dict, plot_vars)
    s2p_plot_rois_funcs.time_series_plot(s2p_data_dict, path_dict, plot_vars)

    #function that converts the s2p output files into csv and npy files for further post-processing
    s2p_con_func.csv_npy_save(fdir, fname, threshold_scaling_values)

{'data_path': ['C:\\Users\\Alex_dl\\Documents\\GitHub\\NAPE_imaging_postprocess\\napeca_post\\sample_data']}
tif
** Found 1 tifs - converting to binary **
2000 frames of binary, time 5.65 sec.
4000 frames of binary, time 10.38 sec.


KeyboardInterrupt: 

In [9]:
s2p_con_func.csv_npy_save(fdir, fname, 0.8)